In [3]:
import pandas as pd

In [4]:
data_source_1_path = "./sources/ETLDataSource1.xlsx"
data_source_2_path = "./sources/ETLDataSource2.xlsx"

# Load Data

In [5]:
orderSource1 = pd.read_excel(data_source_1_path, sheet_name="orderSource1")
productSource1 = pd.read_excel(data_source_1_path, sheet_name="productSource1")

orderSource2 = pd.read_excel(data_source_2_path, sheet_name="orderSource2")
productSource2 = pd.read_excel(data_source_2_path, sheet_name="productSource2")

stateLookUp = pd.read_excel(data_source_1_path, sheet_name="StateLookup")

# Transform Source 1

In [6]:
# Join order and product with OrderID
data_source_1 = pd.merge(orderSource1, productSource1, on="OrderID", how="inner")

# Replace Dictionary
state_dict = dict(zip(stateLookUp["Abbreviation"], stateLookUp["State"]))
data_source_1 = data_source_1.replace({"CustomerState": state_dict})

# Split CustomerName
# Rename CustomerName[0] to CustomerFirstName and CustomerName[0] to CustomerLastName
data_source_1[['CustomerFirstName','CustomerLastName']] = data_source_1["CustomerName"].str.split(" ", n = 1, expand = True)

data_source_1 = data_source_1.drop(columns=["CustomerName"])

# Reorder Attributes alphabetically ascending
data_source_1 = data_source_1.reindex(sorted(data_source_1.columns), axis=1)

# Transform Source 2

In [7]:
data_source_2 = pd.merge(orderSource2, productSource2, on="OrderID", how="inner")

# Replace OrderID prefix A with ""
data_source_2["OrderID"] = data_source_2["OrderID"].str.replace("A", "")

# Parse OrderId to int
data_source_2["OrderID"] = data_source_2["OrderID"].astype(int)

# Change CustomerStatus type to polynomial
data_source_2["CustomerStatus"] = data_source_2["CustomerStatus"].astype("category").cat.codes

CustomerStatus_map = {
    0: "Silver",
    1: "Gold",
    2: "Platinum"
}

# Replace CustomerStatus with map
data_source_2["CustomerStatus"] = data_source_2["CustomerStatus"].replace(CustomerStatus_map)

# Remove totalDiscount column
data_source_2 = data_source_2.drop(columns=["TotalDiscount"])

# Generate new column TotalDiscount
data_source_2["TotalDiscount"] = data_source_2["FullPrice"] - data_source_2["ExtendedPrice"]

# Reorder Attributes alphabetically ascending
data_source_2 = data_source_2.reindex(sorted(data_source_2.columns), axis=1)

# Append and write to Excel

In [12]:
data_source = pd.concat([data_source_1, data_source_2], ignore_index=True)

# Generate concatenated column customerName from CustomerFirstName and CustomerLastName
data_source["customerName"] = data_source["CustomerFirstName"] + "_" + data_source["CustomerLastName"]

# Remove CustomerFirstName and CustomerLastName
data_source = data_source.drop(columns=["CustomerFirstName", "CustomerLastName"])

# Reorder Attributes alphabetically ascending
data_source = data_source.reindex(sorted(data_source.columns), axis=1)

# Write to Excel
data_source.to_excel("./Result_Python.xlsx", index=False)

data_source

,CustomerCity,CustomerState,CustomerStatus,Discount,ExtendedPrice,FullPrice,OrderDate,OrderID,Product,ProductID,Quantity,TotalDiscount,UnitPrice,customerName
0,Pittsburgh,Pennsylvania,Silver,0.00,168.00,168.0,2011-01-11,10248,Queso Cabrales,11,12,0.00,14.0,Suzan_Plock
1,Pittsburgh,Pennsylvania,Silver,0.00,98.00,98.0,2011-01-11,10248,Singaporean Hokkien Fried Mee,42,10,0.00,9.8,Suzan_Plock
2,Pittsburgh,Pennsylvania,Silver,0.00,174.00,174.0,2011-01-11,10248,Mozzarella di Giovanni,72,5,0.00,34.8,Suzan_Plock
3,Miami,Florida,Platinum,0.00,1696.00,1696.0,2011-01-11,10249,Manjimup Dried Apples,51,40,0.00,42.4,Allan_Strate
4,Miami,Florida,Platinum,0.00,167.40,167.4,2011-01-11,10249,Tofu,14,9,0.00,18.6,Allan_Strate
5,Philadelphia,Pennsylvania,Platinum,0.15,1261.40,1484.0,2011-01-11,10250,Manjimup Dried Apples,51,35,222.60,42.4,Elnora_Willison
6,Philadelphia,Pennsylvania,Platinum,0.00,77.00,77.0,2011-01-11,10250,Jack's New England Clam Chowder,41,10,0.00,7.7,Elnora_Willison
7,Philadelphia,Pennsylvania,Platinum,0.15,214.20,252.0,2011-01-11,10250,Louisiana Fiery Hot Pepper Sauce,65,15,37.80,16.8,Elnora_Willison
8,Tacoma,Washington,Platinum,0.00,336.00,336.0,2011-01-11,10251,Louisiana Fiery Hot Pepper Sauce,65,20,0.00,16.8,Daniela_Becknell
9,Tacoma,Washington,Platinum,0.05,95.76,100.8,2011-01-11,10251,Gustaf's Knäckebröd,22,6,5.04,16.8,Daniela_Becknell
